In [1]:
from load_interactions import load_interactions
from build_user_post_graph import build_user_post_graph
from train_lightgcn import train_lightgcn
from recommend_for_user import recommend_for_user
from temporal_split import create_temporal_split
from evaluate import evaluate_recommendations

import pandas as pd
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

############################################
# 5. Main: Putting it all together
############################################
# Load interactions
# likes_df, follows_df, posts_df = load_interactions()
#likes_df, follows_df, _ = load_interactions()
likes_df = load_interactions()

# Create temporal split
# change split_ratio to change the split timestamp
train_data, test_data, test_interactions = create_temporal_split(likes_df, split_ratio=0.8)

# # Build bipartite user-post graph
# data = build_user_post_graph(likes_df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Loaded 22131398 likes before Jun 14, 2023
Number of unique users: 83884, number of unique posts: 4778546
Number of unique users: 14041, number of unique posts: 36763
Train interactions: 17705118
Test interactions: 89474
Split timestamp: 2023-06-21 21:20:10.262000
Users with test interactions: 14041


In [1]:
import duckdb

con = duckdb.connect('/home/sgan/scan_results.duckdb')

# Get likes with URI format
lookback_df = con.execute("""
    SELECT 
        'at://' || repo || '/app.bsky.feed.like/' || rkey as interaction_uri,
        json_extract_string(record, '$.subject.uri') as post_uri,
        TRIM(
            LEADING '/' FROM 
            TRIM(
                TRAILING '"' FROM 
                CONCAT(
                    REPLACE(SUBSTR(json_extract(record, '$.subject.uri')::STRING, 6), '/app.bsky.feed.post/', '_')
                )
            ) 
        ) AS formatted_postUri,
        repo as user_uri,
        createdAt as timestamp
    FROM records 
    WHERE collection = 'app.bsky.feed.like'
        AND createdAt >= '2023-06-23 19:47:26'
        AND createdAt < '2023-06-23 21:47:26'
""").fetchdf()

# Get likes with URI format
test_df = con.execute("""
    SELECT 
        'at://' || repo || '/app.bsky.feed.like/' || rkey as interaction_uri,
        json_extract_string(record, '$.subject.uri') as post_uri,
        TRIM(
            LEADING '/' FROM 
            TRIM(
                TRAILING '"' FROM 
                CONCAT(
                    REPLACE(SUBSTR(json_extract(record, '$.subject.uri')::STRING, 6), '/app.bsky.feed.post/', '_')
                )
            ) 
        ) AS formatted_postUri,
        repo as user_uri,
        createdAt as timestamp
    FROM records 
    WHERE collection = 'app.bsky.feed.like'
        AND createdAt >= '2023-06-23 21:47:26'
        AND createdAt < '2023-06-23 23:47:26'
""").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [2]:
lookback_df_post = set(lookback_df['formatted_postUri'])
test_df_post = set(test_df['formatted_postUri'])
len(test_df_post - lookback_df_post) / len(test_df_post)

0.795379054318093

In [4]:
len(lookback_df_post)

22190

In [1]:
from load_interactions import load_interactions
from build_user_post_graph import build_user_post_graph, build_user_post_graph_v2
from train_lightgcn import train_lightgcn
from recommend_for_user import recommend_for_user
from temporal_split import create_temporal_split, create_split_with_time
from evaluate import evaluate_recommendations

import pandas as pd
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

############################################
# 5. Main: Putting it all together
############################################
# Load interactions
# likes_df, follows_df, posts_df = load_interactions()
likes_df = load_interactions()

# train_df, val_df, test_df = create_split_with_time(likes_df)

# def build_combined_graph(train_df, val_df, test_df=None):
#     # Combine edges from train, val, and test dataframes
#     combined_df = pd.concat([train_df, val_df, test_df]).drop_duplicates()
#     return build_user_post_graph_v2(combined_df)

combined_data = build_user_post_graph_v2(likes_df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Loaded 22131398 likes before Jun 14, 2023


/home/sgan/private/DyGLib/pytorch_lightgcn/build_user_post_graph.py:86: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2.02304252e+13 2.02304302e+13 2.02305011e+13 ... 2.02306222e+13
 2.02306222e+13 2.02304272e+13]' has dtype incompatible with datetime64[us], please explicitly cast to a compatible dtype first.
  likes_df.loc[:,'timestamp'] = likes_df['timestamp'].apply(convert_timestamp_to_float)


Number of unique users: 106367, number of unique posts: 5866226
44260392


In [2]:
combined_data.num_users

106367

In [3]:
print(f"Max index in edge_index: {combined_data.edge_index.max()}")
print(f"Number of nodes (data.num_nodes): {combined_data.num_nodes}")
print(f"Min index in edge_index: {combined_data.edge_index.min()}")

Max index in edge_index: 5972592
Number of nodes (data.num_nodes): 5972593
Min index in edge_index: 0


In [4]:
train_edge_index = combined_data.edge_index[:, combined_data.train_mask]
val_edge_index = combined_data.edge_index[:, combined_data.val_mask]
test_edge_index = combined_data.edge_index[:, combined_data.test_mask]

In [5]:
from torch_geometric.data import Data
train_data = Data(
    edge_index=train_edge_index,
    num_nodes=combined_data.num_nodes
)
train_data.num_users = combined_data.num_train_users
train_data.num_items = combined_data.num_train_posts

In [6]:
from train_lightgcn import train_lightgcn_v2

model = train_lightgcn_v2(train_data, embedding_dim=32, num_layers=1, epochs=1)

# Get number of nodes and edges
# print(f"Number of nodes: {train_data.num_nodes}. Number of users: {train_data.num_users}. Number of posts: {train_data.num_items}")
# print(f"Number of edges: {train_data.num_edges}. Number of likes: {len(likes_df)}") # remember edges are doubled then coalesced (deduplicated, sorted, etc.)

Epoch 01: 100%|██████████| 946/946 [02:55<00:00,  5.40it/s, batch_loss=0.0648]


Epoch 01 Summary:
  Average Loss: 0.0976
  Min Batch Loss: 0.0580
  Max Batch Loss: 0.6931
  Loss Std Dev: 0.0882



In [7]:
embeddings = model.get_embedding(combined_data.edge_index.to(device))

# Split embeddings into user and post embeddings
user_embeddings = embeddings[:combined_data.num_users]
post_embeddings = embeddings[combined_data.num_users:]

val_edge_index = combined_data.edge_index[:, combined_data.val_mask]
test_edge_index = combined_data.edge_index[:, combined_data.test_mask]

# Separate validation users and posts based on ID range
val_user_ids = torch.unique(val_edge_index[val_edge_index < combined_data.num_users])  # IDs < num_users
val_post_ids = torch.unique(val_edge_index[val_edge_index >= combined_data.num_users])  # IDs >= num_users

# Separate test users and posts based on ID range
test_user_ids = torch.unique(test_edge_index[test_edge_index < combined_data.num_users])  # IDs < num_users
test_post_ids = torch.unique(test_edge_index[test_edge_index >= combined_data.num_users])  # IDs >= num_users

In [23]:
embeddings.shape

torch.Size([5972593, 32])

In [31]:
combined_data.num_users + combined_data.num_items

val_edge_index

tensor([[      1,       1,       1,  ..., 5972588, 5972589, 5972590],
        [ 107131,  107132,  107133,  ...,  106364,  106364,  106364]])

In [16]:
val_edge_index = combined_data.edge_index[:, combined_data.val_mask]
test_edge_index = combined_data.edge_index[:, combined_data.test_mask]
# Validation users and posts
val_user_ids = torch.unique(val_edge_index[0])  # Users are in the first row
val_post_ids = torch.unique(val_edge_index[1])  # Posts are in the second row

# Test users and posts
test_user_ids = torch.unique(test_edge_index[0])  # Users are in the first row
test_post_ids = torch.unique(test_edge_index[1])  # Posts are in the second row

# Validation embeddings
val_user_embeddings = user_embeddings[val_user_ids]
val_post_embeddings = post_embeddings[val_post_ids - combined_data.num_train]  # Subtract offset for posts

# Test embeddings
test_user_embeddings = user_embeddings[test_user_ids]
test_post_embeddings = post_embeddings[test_post_ids - combined_data.num_users]  # Subtract offset for posts


../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [3073,0,0], thread: [0,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [3073,0,0], thread: [1,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [3073,0,0], thread: [2,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [3073,0,0], thread: [3,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [3073,0,0], thread: [4,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [3073,0,0], thread: [5,0,0] Assertion 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [2]:
model = train_lightgcn(train_data, embedding_dim=32, num_layers=1, epochs=1)

# Get number of nodes and edges
print(f"Number of nodes: {train_data.num_nodes}. Number of users: {train_data.num_users}. Number of posts: {train_data.num_items}")
print(f"Number of edges: {train_data.num_edges}. Number of likes: {len(likes_df)}") # remember edges are doubled then coalesced (deduplicated, sorted, etc.)

Epoch 01: 100%|██████████| 1081/1081 [03:24<00:00,  5.28it/s, batch_loss=0.0849]


Epoch 01 Summary:
  Average Loss: 0.1060
  Min Batch Loss: 0.0711
  Max Batch Loss: 0.6931
  Loss Std Dev: 0.0830

Number of nodes: 4862430. Number of users: 83884. Number of posts: 4778546
Number of edges: 35407992. Number of likes: 22131398


In [3]:
# Evaluate the model
recall_scores = evaluate_recommendations(model, test_interactions, test_data, k_values=[20, 100, 500,1000, 5000, 10000, 20000])

# Print results
print("\nEvaluation Results:")
print("-" * 50)
print("Recall@K Scores:")
for k, score in recall_scores.items():
    print(f"Recall@{k:5d}: {score:.4f}")
print("-" * 50)

# Print some statistics about the test set
num_users = len(test_interactions)
num_posts = sum(len(posts) for posts in test_interactions.values())
avg_posts = num_posts / num_users

print(f"\nTest Set Statistics:")
print(f"Number of users: {num_users}")
print(f"Total posts: {num_posts}")
print(f"Average posts per user: {avg_posts:.2f}")

# Print distribution of posts per user
post_counts = [len(posts) for posts in test_interactions.values()]
print(f"\nPost distribution:")
print(f"Min posts per user: {min(post_counts)}")
print(f"Max posts per user: {max(post_counts)}")

100%|██████████| 56/56 [00:23<00:00,  2.38it/s]


Evaluation Results:
--------------------------------------------------
Recall@K Scores:
Recall@   20: 0.0085
Recall@  100: 0.0367
Recall@  500: 0.1337
Recall@ 1000: 0.2231
Recall@ 5000: 0.6343
Recall@10000: 0.8576
Recall@20000: 1.0000
--------------------------------------------------

Test Set Statistics:
Number of users: 7062
Total posts: 38186
Average posts per user: 5.41

Post distribution:
Min posts per user: 1
Max posts per user: 382


In [4]:
# Get training embeddings
train_embeddings = model.get_embedding(train_data.edge_index.to(device))
print("Training embeddings shape:", train_embeddings.shape)

# Get test embeddings - but only for nodes that appear in test edges
test_nodes = torch.unique(test_data.edge_index)
test_embeddings = model.get_embedding(test_data.edge_index.to(device))
test_embeddings = test_embeddings[test_nodes]
print("Test embeddings shape:", test_embeddings.shape)

Training embeddings shape: torch.Size([846526, 32])
Test embeddings shape: torch.Size([22953, 32])


In [5]:
user_idx_example = 1014
user_did, user_profile_url, recommendations, rec_content = recommend_for_user(model, user_idx_example, train_data, top_k=20)

# Get user's recent likes
user_likes = likes_df[likes_df['user_uri'] == user_did].sort_values('timestamp', ascending=False).head(5)

print(f"Recommendations for user {user_did}")
print(f"User profile: {user_profile_url}")

print("\nUser's 5 most recent likes:")
print("-" * 50)
for _, like in user_likes.iterrows():
    post_uri = like['post_uri']
    timestamp = like['timestamp']
    # Convert AT URI to web URL
    parts = post_uri.split('/')
    post_did = parts[2]
    post_id = parts[-1]
    web_url = f"https://bsky.app/profile/{post_did}/post/{post_id}"
    print(f"Liked on: {timestamp}")
    print(f"Post: {web_url}\n")

print("\nRecommended posts:")
print("-" * 50)
for i, (at_uri, web_url) in enumerate(recommendations, 1):
    content = rec_content.get(at_uri, {'text': 'Post not found', 'author': 'Unknown', 'created_at': 'Unknown'})
    print(f"\n{i}. By @{content['author']}")
    print(f"   {web_url}")
    print(f"   Posted: {content['created_at']}")
    print(f"   Text: {content['text'][:200]}...")  # Truncate long posts

Recommendations for user did:plc:e2a2sywvv7dwlu4h4qdt3h7j
User profile: https://bsky.app/profile/did:plc:e2a2sywvv7dwlu4h4qdt3h7j

User's 5 most recent likes:
--------------------------------------------------
Liked on: 2023-04-29 03:02:05.147000
Post: https://bsky.app/profile/did:plc:oideekcq5mb76zg6sc5ntbly/post/3jui4a5d7lu2m

Liked on: 2023-04-18 03:21:07.003000
Post: https://bsky.app/profile/did:plc:vpkhqolt662uhesyj6nxm7ys/post/3jtmca54xqs26

Liked on: 2023-04-18 03:19:31.663000
Post: https://bsky.app/profile/did:plc:ukxj2up3soemlxv26h6s6tfb/post/3jtmg6xyuw526

Liked on: 2023-03-24 18:24:15.454000
Post: https://bsky.app/profile/did:plc:t2xgjn25va5r5m3prh6ssuhi/post/3jrkujpel4c22

Liked on: 2023-03-15 17:47:39.734000
Post: https://bsky.app/profile/did:plc:7yhhddnpipsbj57ybzduaov3/post/3jqvovw6o432s


Recommended posts:
--------------------------------------------------

1. By @did:plc:ragtjsm2j2vknwkz3zp4oxrd
   https://bsky.app/profile/did:plc:ragtjsm2j2vknwkz3zp4oxrd/post/3jtqhia

In [6]:
# Get URLs and stats for some 30 users
inv_user2id = {v: k for k, v in train_data.user2id.items()}
for user_idx in range(1000, 1030):  # change this to whatever you want to look at different users
    user_did = inv_user2id[user_idx]  # user_uri is already in DID format
    profile_url = f"https://bsky.app/profile/{user_did}"
    
    # Get number of likes and following
    user_likes_count = len(likes_df[likes_df['user_uri'] == user_did])
    user_following_count = len(train_data.edge_index[0][train_data.edge_index[0] == user_idx])
    
    print(f"{user_idx}: {profile_url}")
    print(f"    Likes: {user_likes_count}")
    print(f"    Following: {user_following_count}")

1000: https://bsky.app/profile/did:plc:siu4mwwnhd32x3n4ci2xytgt
    Likes: 217
    Following: 217
1001: https://bsky.app/profile/did:plc:zpkygt5hct4ye6tlmusv5mvb
    Likes: 59
    Following: 59
1002: https://bsky.app/profile/did:plc:u4mi54yplxzlrtdgubbs2r7k
    Likes: 221
    Following: 221
1003: https://bsky.app/profile/did:plc:ksyu6kwmhge4uh3nxnn3o7c4
    Likes: 399
    Following: 393
1004: https://bsky.app/profile/did:plc:46375o72emh22j3bgeltmw4x
    Likes: 11
    Following: 11
1005: https://bsky.app/profile/did:plc:kz5wsopyrb2o6vlh2lkjt4v7
    Likes: 32
    Following: 32
1006: https://bsky.app/profile/did:plc:dyjm3keeut27qu4sno7ow7ry
    Likes: 495
    Following: 471
1007: https://bsky.app/profile/did:plc:2gj5zzmaa5pgc3c7hv334j5l
    Likes: 99
    Following: 97
1008: https://bsky.app/profile/did:plc:tz45rben2kkv2vcahflgdtlt
    Likes: 22
    Following: 19
1009: https://bsky.app/profile/did:plc:nu5ivyol6dytbro5o2yxqo5d
    Likes: 313
    Following: 311
1010: https://bsky.app/profile

In [11]:
def find_similar_users(model, user_idx, train_data, top_k=5):
    # Get all embeddings
    embeddings = model.get_embedding(train_data.edge_index.to(device))
    
    # Get user embeddings only
    user_embeddings = embeddings[:train_data.num_users]
    
    # Calculate cosine similarity between target user and all other users
    target_embedding = user_embeddings[user_idx].unsqueeze(0)
    cos = torch.nn.CosineSimilarity(dim=1)
    similarities = cos(target_embedding, user_embeddings)
    
    # Get top-k similar users (excluding the user themselves)
    similarities[user_idx] = -1  # Exclude self
    top_similar = torch.topk(similarities, k=top_k)
    
    # Convert user indices back to DIDs
    inv_user2id = {v: k for k, v in train_data.user2id.items()}
    similar_users = []
    
    # Add .detach() before .numpy()
    for idx, score in zip(top_similar.indices.cpu().detach().numpy(), 
                         top_similar.values.cpu().detach().numpy()):
        user_did = inv_user2id[idx]
        profile_url = f"https://bsky.app/profile/{user_did}"
        similar_users.append((profile_url, score))
    
    return similar_users

# Example usage:
user_idx_example = 1014  # You can change this to any user index
similar_users = find_similar_users(model, user_idx_example, train_data, top_k=5)

print(f"User profile: {user_profile_url}")
print(f"Similar users to user index {user_idx_example}:")
print("-" * 50)
for profile_url, similarity_score in similar_users:
    print(f"Similarity score: {similarity_score:.4f}")
    print(f"Profile: {profile_url}\n")

User profile: https://bsky.app/profile/did:plc:e2a2sywvv7dwlu4h4qdt3h7j
Similar users to user index 1014:
--------------------------------------------------
Similarity score: 0.7648
Profile: https://bsky.app/profile/did:plc:sqvt4jbt6tjjarugjlw5ycnn

Similarity score: 0.7344
Profile: https://bsky.app/profile/did:plc:yp255lkwkcal42wd55ywfum2

Similarity score: 0.7071
Profile: https://bsky.app/profile/did:plc:lipqor2zk6q6sagskv5pzees

Similarity score: 0.7069
Profile: https://bsky.app/profile/did:plc:ns7pzrboi4swrz7gpacdcia3

Similarity score: 0.7027
Profile: https://bsky.app/profile/did:plc:jg6ouvyo7inrrxvllqqelkao

